In [10]:
import numpy as np
import pandas as pd
import rao_utilities as util
import rao_plotting as plot
import rao_models as model
import rao_math as m
import warnings
import importlib

from numpy import sqrt
from feets import ExtractorWarning
from pathlib import PurePath

importlib.reload(util)
importlib.reload(plot)
importlib.reload(model)
importlib.reload(m)

## CONSTANTS ============================================================================
# FILENAME = 'data/data.csv'
# FILENAME = 'data/M3_raw01_Photometry25-47-59.csv' 
# FILENAME = 'data/M3_raw02_Photometry25-47-59.csv'
# FILENAME = 'data/M3_raw03_Photometry28-47-55.csv'
# FILENAME = 'data/M3_raw04_Photometry35-47-55.csv'
# FILENAME = 'data/M3_raw01_Photometry28_47_55_may_29.csv' # Change me to work with different files/filename
# FILENAME = 'data/M3_2nights_rawPhotometry.csv'
FILENAME = 'data/M3_night_1.xlsx'
# FILENAME = 'data/M3_night_2.xlsx'
EXPOSURE_TIME = 2.5

warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning) # Need this to prevent it from spamming
warnings.filterwarnings("ignore", category=ExtractorWarning)
## FUNCTIONS ============================================================================

# I moved everything into different python modules since this was getting extremely large

## CALCULATION ============================================================================
file = PurePath(FILENAME)

df = util.extract_data(FILENAME)
bad_rows = df[df['name'] == 'M3-12']
df = df.drop(index=bad_rows.index)

mags = df['mag'].to_numpy(dtype='float64')
error = df['error'].to_numpy(dtype='float64')

num_stars, samples = util.extract_samples_stars(df)
if 'jd' in df.columns:
    timeline = df['jd'].unique()
    df['jd'] = pd.to_datetime(df['jd'], origin='julian', unit='D')
else: 
    timeline = np.linspace(0, samples*EXPOSURE_TIME, samples)

mags = np.reshape(mags, (samples, num_stars))      # Stack both mags and error according to time, new column = new file in time
error = np.reshape(error, (samples, num_stars))
diff_mags = m.subtract_all_magnitudes(mags)          # array of arrays, each array corresponding to a single star as the 'target'
uncertainties = m.calculate_all_uncertainties(error) # Each individual array is organized by incrementing time in columns, rows are stars
average_diff_mags = np.mean(diff_mags, axis=1, dtype=np.float64) # mean of every column, accumulating all stars at particular time
average_uncertainties = sqrt(np.sum(uncertainties**2, axis=1, dtype=np.float64))/uncertainties[0].shape[0] # sum of squares of each column, then sqrt'd

df['average_diff_mags'] = average_diff_mags.T.reshape(samples*num_stars, 1)
df['average_uncertainties'] = average_uncertainties.T.reshape(samples*num_stars, 1)

box_model_results = model.box_least_squares(average_diff_mags, average_uncertainties, timeline=timeline)

# # Feature Extraction information
# indices = m.calculate_all_feets_indices(data = average_diff_mags, 
#                                           timeline = timeline, 
#                                           uncertainties= average_uncertainties)
# feature_df = pd.DataFrame(indices)
# feature_df.to_excel('Features_of_stars.xlsx')

# # Data export for Dr. Langill
# indices_of_stars = [ sub['name'] for sub in box_model_results]
# original_data_filtered = df.iloc[indices_of_stars]
# original_data_filtered.to_excel('Stars_with_varying_light.xlsx')

# # Chi squared test
chi = model.reduced_chi_square(mags.T, error.T, timeline)
print(chi)
# pd.DataFrame({'chisquared':chi}).to_excel('chi_squared_test.xlsx')

# plot.plot_and_save_all(df, file.stem)

[1.60057088e+01 6.88166470e+00 4.29575870e+01 7.65119954e-01
 7.88758763e+00 1.68495764e+00 2.94945571e+00 3.53158301e+00
 4.70316542e-01 7.70918462e+00 4.21756171e+01 2.63958383e-01
 5.96930500e+00 4.19386495e+00 3.34564289e-01 8.69532962e+01
 2.22737031e+00 3.54608165e+00 4.18782637e-01 9.92444388e-01
 2.72634532e+00 2.46127011e+00 4.36215040e-01 3.09111743e-01
 1.55771788e+00 8.89290164e-01 7.56153358e-01 1.28644674e+00
 1.69999584e+00 6.12119759e-01 8.45751084e-01 6.90075255e-01
 1.96016118e+00 1.25581031e+00 1.64543118e+00 2.77588548e+00
 1.46590834e+00 4.01660871e-02 1.11673123e+00 2.25623521e+00
 2.02991975e+00 1.54443616e+00 6.88916030e-01 1.71685842e+00
 1.40741833e+00 2.85933956e-01 3.65819254e-01 1.98323326e+00
 2.97112486e-02 4.71115023e-02 1.98369711e-01 6.84578710e-02
 1.41310767e+00 3.60276280e-01 4.28017690e-01 1.40914294e+00
 3.62484792e-01 1.06693216e+00 5.71020592e-01 3.97677672e-01
 5.60224838e-01 2.98087217e-01 3.04385174e+00 2.70768096e+00
 1.06863336e+00 4.350450